In [1]:
from models.DemoV3 import DemoNet
from utils.config import GlobalConfig
from utils.datasets.carla_data import CARLA_Data
from torch.utils.data import DataLoader

ModuleNotFoundError: No module named 'models.DemoV3'

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import torch
import torch.nn.functional as F
from torch import nn
import os

In [ ]:
config = GlobalConfig()

In [ ]:
model = DemoNet(config, 'cuda')
model = nn.DataParallel(model, device_ids=[0]).cuda()

In [ ]:
model.load_state_dict(torch.load(os.path.join('/root/End2End/log/Demov1/demo', 'best_model.pth')))
model.eval()
model = model.module

In [ ]:
val_set = CARLA_Data(root=config.val_data, config=config)
dataloader_val = DataLoader(val_set, batch_size=4, shuffle=False, num_workers=1, pin_memory=True)


In [ ]:
dataloader_val

In [ ]:
for data in dataloader_val:
	#data = val_set.__getitem__(233)
	fronts_in = data['fronts']
	lefts_in = data['lefts']
	rights_in = data['rights']
	fronts = []
	lefts = []
	rights = []
	for i in range(config.seq_len):
		fronts.append(fronts_in[i].to(config.device, dtype=torch.float32))
		if not config.ignore_sides:
			lefts.append(lefts_in[i].to(config.device, dtype=torch.float32))
			rights.append(rights_in[i].to(config.device, dtype=torch.float32))
		if not config.ignore_rear:
			rears.append(rears_in[i].to(config.device, dtype=torch.float32))
		# lidars.append(lidars_in[i].to(args.device, dtype=torch.float32))
	break

In [ ]:
fronts = torch.cat(fronts, dim=1).unsqueeze(1)
lefts = torch.cat(lefts, dim=1).unsqueeze(1)
rights = torch.cat(rights, dim=1).unsqueeze(1)
image = torch.cat((fronts, lefts, rights), dim=1)

In [ ]:
fronts.shape

In [ ]:
seg_front = data['seg_fronts']

In [ ]:
plt.rcParams['figure.figsize'] = (16.0, 6.0) 
# f, axarr = plt.subplots(1, 1)
# axarr[0].imshow(lefts[0,0,:,:,:].detach().cpu().permute(1, 2, 0).int(), cmap='Greys_r')
plt.imshow(seg_front[0].detach().cpu().int(), cmap='Greys_r')
# axarr[2].imshow(rights[0,0,:,:,:].detach().cpu().permute(1, 2, 0).int(), cmap='Greys_r')
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = (16.0, 6.0) 
# f, axarr = plt.subplots(1, 1)
# axarr[0].imshow(lefts[0,0,:,:,:].detach().cpu().permute(1, 2, 0).int(), cmap='Greys_r')
plt.imshow(fronts[0][:,:,:].detach().cpu().permute(1, 2, 0).int(), cmap='Greys_r')
# axarr[2].imshow(rights[0,0,:,:,:].detach().cpu().permute(1, 2, 0).int(), cmap='Greys_r')
plt.show()

In [ ]:
command = torch.tensor(data['command'])
gt_velocity = torch.tensor(data['velocity'])
gt_steer = torch.tensor(data['steer'])
gt_throttle = torch.tensor(data['throttle'])
# gt_brake = data['brake'].to(args.device, dtype=torch.float32)
gt_brake = torch.tensor(data['brake'])
target_brake = []

if gt_brake:
	target_brake.append(F.one_hot(torch.tensor(1), num_classes=2))
else:
	target_brake.append(F.one_hot(torch.tensor(0), num_classes=2))
# target point

# target_brake = torch.stack(target_brake, dim=0).to('cuda', dtype=torch.float32)

# target point
# target_point = torch.stack(data['target_point'], dim=1).to('cuda', dtype=torch.float32)

car_status = torch.stack((gt_velocity, command), dim=0).to('cuda', dtype=torch.float32)
target_ctr = torch.stack((gt_steer, gt_throttle), dim=0).to('cuda', dtype=torch.float32)
target_ctr = torch.cat((target_ctr, target_brake[0].to('cuda'))).to('cuda', dtype=torch.float32)


In [ ]:
image = image.unsqueeze(0)


In [ ]:
b, s, c, w, h = image.shape

image = image.view(-1, c, w, h)
for i in range(b * s):

    image[i,0,:,:] = (image[i,0,:,:] - 0.485) / 0.229
    image[i,1,:,:] = (image[i,1,:,:] - 0.456) / 0.224
    image[i,2,:,:] = (image[i,2,:,:] - 0.406) / 0.225

image = image.view(b, s, c, w, h)
left = image[:, 0, :, :, :]
center = image[:, 1, :, :, :]
right = image[:, 2, :, :, :]
# image = image.view(-1, c, w, h)
left = model.image_encode_net_left(left)
center = model.image_encode_net_center(center)
right = model.image_encode_net_right(right)   

image = torch.cat((left, center, right), dim=1)
control = model.pred_ctr_net(image, car_status)

In [ ]:
control

In [ ]:
pred_ctr

In [ ]:
target_ctr

In [ ]:
car_status

In [ ]:
plt.rcParams['figure.figsize'] = (16.0, 6.0) 
f, axarr = plt.subplots(2, 6)
for i in range(12):
    axarr[i//6][i%6].imshow(fronts[0,i,:,:,:].detach().cpu().permute(1, 2, 0).int(), cmap='Greys_r')

plt.show()

In [ ]:
sum_img_features_total = img_features_total.squeeze(0).sum(dim=1)

In [ ]:
[sum_img_features_total[i, :, :].max() for i in range(12)]

In [ ]:
plt.rcParams['figure.figsize'] = (16.0, 6.0) 
f, axarr = plt.subplots(2, 6)
for i in range(12):
    pic1 = sum_img_features_total[i, :, :]
    pic1 = pic1.squeeze(0).detach().cpu()
    axarr[i//6][i%6].imshow(pic1, cmap='Greys_r')

plt.show()


In [ ]:
plt.rcParams['figure.figsize'] = (16.0, 6.0) 
f, axarr = plt.subplots(1,6)
for i in range(6):
    pic1 = post_feature_weight.sum(dim=2)[0, i, :, :]
    pic1 = pic1.squeeze(0).detach().cpu()
    # plt.imshow(pic1[:, :], cmap='Greys_r')
    axarr[i].imshow(pic1, cmap='Greys_r')

plt.show()

In [ ]:
pred_features.shape

In [ ]:
plt.rcParams['figure.figsize'] = (16.0, 6.0)
f, axarr = plt.subplots(1, 6)
for i in range(6):
    pic1 = pred_features[:, :, 0, :, :][0, i, :, :]
    pic1 = pic1.squeeze(0).detach().cpu()
    # plt.imshow(pic1[:, :], cmap='Greys_r')
    axarr[i].imshow(pic1, cmap='Greys_r')

plt.show()

In [ ]:
[pred_w[0, i].mean() for i in range(6)]

In [ ]:
plt.rcParams['figure.figsize'] = (16.0, 6.0) 
f, axarr = plt.subplots(1,6)
for i in range(6):
    pic1 = pred_features_weight.sum(dim=2)[0, i, :, :]
    pic1 = pic1.squeeze(0).detach().cpu()
    # plt.imshow(pic1[:, :], cmap='Greys_r')
    axarr[i].imshow(pic1, cmap='Greys_r')

plt.show()

In [ ]:
train_loss = [0.25132067886491616, 0.09826476913655088, 0.09170093669423035, 0.08573785524017044, 0.0811269340061006, 0.07812297730152273, 0.07653789484562973, 0.07659081295576124, 0.07478648886261952, 0.07437606727527012, 0.07411234903348876, 0.07329118151572489, 0.07273593382865545, 0.0716391432143393, 0.07417503932402247, 0.07171010868623853, 0.06992584528578889, 0.06789819498856862, 0.07019227030243547, 0.0672841942115199, 0.06630738536782918, 0.06300994645405028, 0.06107551265685331, 0.0552142395992719, 0.050358380596818665, 0.05533859730875563, 0.047644188944693834, 0.04159163324428456, 0.03886260759706298, 0.03648236599484725, 0.03215596163423643, 0.029095376249668854, 0.027690931857519207, 0.026536207952137503, 0.02684093692029516, 0.023441429198941304, 0.022250035083630965, 0.021509626793808172, 0.02014488194115637, 0.01941306068162833, 0.019818898599727876, 0.01858583208661349, 0.017627106422913215, 0.015140145214363222, 0.014094731236054075, 0.012817011148269688, 0.0114042522485501, 0.01068047543743714, 0.010156407329209504, 0.010509116521903446, 0.009366032819352334, 0.00951327770238831, 0.01235562184537273, 0.014063643770558493, 0.010245018531124861, 0.00842660373420499, 0.0075057228129091005, 0.006963801406700874, 0.006902783755434766, 0.006857799014098765, 0.006432314494269944, 0.0060688914343093835, 0.006697583385920596, 0.01094817188679285, 0.009374456948024176, 0.010039020931747343, 0.0071625109662168795, 0.006238600646611303, 0.005840029213244894, 0.0061612515743555765, 0.005393317034135439, 0.005367305696875389, 0.0051948252727188875, 0.005042734235203603, 0.004929979676602497, 0.004863250073200712, 0.004704888298597542, 0.004680911748437211, 0.004760930656145016, 0.004820633026871032, 0.00467170736020697, 0.004935642645882797, 0.004731576529697382, 0.0045314221712206264, 0.005343950129601927, 0.011845215544148925, 0.014639477212248104, 0.008037432167856465, 0.008957596239633858, 0.007010514101767469, 0.005941084022301116, 0.004887352285662755, 0.004668386326548422, 0.004613823237429772, 0.005473129257249336, 0.004809911245302785, 0.004509805093541564, 0.004378276405346004, 0.0043959845658502585, 0.004251234229221674, 0.004165552623869319]
val_loss = [0.1051677190299545, 0.08706744532766086, 0.08492286391556263, 0.08631260222090142, 0.08079578736796975, 0.09024366407788226, 0.09153007702635867, 0.10114564459238734, 0.09818041626630085, 0.09596363749088986, 0.0926991729331868, 0.0994164977289204, 0.09267315739499671, 0.09358349019395454, 0.08944728896021843, 0.0881390096220587, 0.0899802293362362, 0.1151176692917943, 0.08968629364722541, 0.08927618109488061, 0.08861237643286586]

In [ ]:
plt.plot(range(len(train_loss)), train_loss, color='red')
plt.plot(range(0, len(train_loss), 5), val_loss, color='blue')


In [ ]:
model = model.to('cuda:0').module

In [ ]:
model.device

In [ ]:
torch.save(model.state_dict(), os.path.join('/root/End2End/log/Demov1/demo', '_best_model.pth'))

In [ ]:
# plt.imshow(fronts[0,5,:,:,:].detach().cpu().permute(1, 2, 0).int())

plt.rcParams['figure.figsize'] = (16.0, 6.0) 
f, axarr = plt.subplots(2, 6)
for i in range(12):
    axarr[i//6][i%6].imshow(fronts[0,i,:,:,:].detach().cpu().permute(1, 2, 0).int(), cmap='Greys_r')

plt.show()